In [2]:
from flask import Flask, render_template, request
import pymysql
from dynaconf import Dynaconf
import flask_login
import requests
import csv
import json
import pandas

In [3]:
#Database connection
conf = Dynaconf(
    settings_file = ["settings.toml"]
)

def connect_db():
    """Connect to the phpMyAdmin database (LOCAL STEAM NETWORK ONLY)"""
    conn = pymysql.connect(
        host = "db.steamcenter.tech",
        database = "apollo",
        user = conf.user,
        password = conf.password,
        autocommit = True,
        cursorclass = pymysql.cursors.DictCursor
    )
    return conn

In [4]:
#Function declaration
def get_data():

    #Variable Declaration
    colleges = []
    page=0

    #Parameters Declaration
    name="school.name"
    tuition="latest.cost.attendance.academic_year"
    admission_rate="latest.admissions.admission_rate.overall"
    average_sat="latest.admissions.sat_scores.average.overall"
    size="student.size"
    city="school.city"
    state="school.state"

    #Inserting parameters into list
    parameters={
        name,
        tuition,
        admission_rate,
        average_sat,
        size,
        city,
        state
    }

    #Converting parameter list into url
    par_str=""
    par_str=",".join(parameters, )


    #While true, collects data from each page
    while True:

        page+=1
        response=requests.get(f"https://api.data.gov/ed/collegescorecard/v1/schools?api_key=fEZsVdtKgtVU4ODIpjHcP8vDttDK0ftSGZaWDcAk&fields={par_str}&page={page}&per_page=100")
        #collects the json of each response
        results = response.json()['results']
        print(results)

        #If the results are empty (the page does not exist), break loop
        if results==[]:
            print("too far")
            break

        #Extend each page result to college list
        colleges.extend(results)
        
    # for page in range(1,2):
        
    #     response=requests.get(f"https://api.data.gov/ed/collegescorecard/v1/schools?api_key=fEZsVdtKgtVU4ODIpjHcP8vDttDK0ftSGZaWDcAk&fields={par_str}&page={page}&per_page=1")
        
    #     results = response.json()['results']

    #     colleges.extend(results)
    
    print("Completed list creation! Inserting data")

    for college in colleges:

        name=college["school.name"]

        tuition=college["latest.cost.attendance.academic_year"]

        admission_rate=college["latest.admissions.admission_rate.overall"]

        average_sat=college["latest.admissions.admission_rate.overall"]

        size=college["student.size"]

        city=college["school.city"]

        state=college["school.state"]

        conn = connect_db()
        cursor = conn.cursor()

        cursor.execute(f"""

        INSERT INTO `Colleges` (`name`, `tuition`, `admission_rate`, `average_sat`, `size`, `city`, `state`)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
                            
                            """, (name, tuition, admission_rate, average_sat, size, city, state))
        
        cursor.close
        conn.close

    return

get_data()

[{'latest.cost.attendance.academic_year': 47865, 'latest.admissions.sat_scores.average.overall': None, 'latest.admissions.admission_rate.overall': 0.7014, 'school.city': 'Glendale', 'school.name': 'Arizona Christian University', 'school.state': 'AZ', 'student.size': None}, {'latest.cost.attendance.academic_year': None, 'latest.admissions.sat_scores.average.overall': None, 'latest.admissions.admission_rate.overall': None, 'school.city': 'Avondale', 'school.name': 'Universal Technical Institute of Arizona Inc', 'school.state': 'AZ', 'student.size': None}, {'latest.cost.attendance.academic_year': 12681, 'latest.admissions.sat_scores.average.overall': None, 'latest.admissions.admission_rate.overall': None, 'school.city': 'Prescott', 'school.name': 'Yavapai College', 'school.state': 'AZ', 'student.size': None}, {'latest.cost.attendance.academic_year': 21300, 'latest.admissions.sat_scores.average.overall': 1089, 'latest.admissions.admission_rate.overall': 0.6146, 'school.city': 'Little Rock'

KeyboardInterrupt: 